In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Đọc dữ liệu
try:
    data = pd.read_csv('Data/lung_data.csv')
except FileNotFoundError:
    print("Không tìm thấy tệp. Vui lòng kiểm tra đường dẫn tệp.")
    exit()

# Chuyển đổi F, M thành số (0 và 1)
data['GENDER'] = data['GENDER'].apply(lambda x: 0 if x == 'F' else 1)

# Chuyển đổi Yes, No thành số (0 và 1) cho các cột nhị phân
binary_columns = ['PEER_PRESSURE', 'FATIGUE', 'ALCOHOL_CONSUMING', 'LUNG_CANCER']
for col in binary_columns:
    data[col] = data[col].apply(lambda x: 1 if x == 'Yes' else 0)

# Kiểm tra dữ liệu
print(data.head())
print(data.info())

# Giả sử cột cuối cùng là nhãn và các cột còn lại là đặc trưng
X = data.iloc[:, :-1].values  # Tất cả các cột trừ cột cuối cùng
y = data.iloc[:, -1].values   # Cột cuối cùng là nhãn

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Xây dựng mô hình mạng nơ-ron nhân tạo
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Lớp đầu vào với số nơ-ron bằng số đặc trưng
model.add(Dropout(0.5))                                             # Dropout để giảm overfitting
model.add(Dense(32, activation='relu'))                             # Lớp ẩn với 32 nơ-ron
model.add(Dense(1, activation='sigmoid'))                           # Lớp đầu ra với 1 nơ-ron và hàm kích hoạt sigmoid cho bài toán nhị phân

# Compile mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2)

# Đánh giá mô hình trên tập kiểm tra
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Độ chính xác trên dữ liệu kiểm tra: {accuracy * 100:.2f}%')


   GENDER  AGE  SMOKING  YELLOW_FINGERS  ANXIETY  PEER_PRESSURE  \
0       1   76        1               2        2              0   
1       0   73        1               2        2              1   
2       1   74        1               1        2              1   
3       0   70        1               2        2              0   
4       0   27        1               1        2              0   

   CHRONIC_DISEASE  FATIGUE  ALLERGY  WHEEZING  ALCOHOL_CONSUMING  COUGHING  \
0                2        1        1         1                  0         2   
1                2        1        1         1                  1         2   
2                1        1        2         2                  1         1   
3                2        1        1         2                  0         2   
4                1        0        2         2                  1         1   

   SHORTNESS_OF_BREATH  SWALLOWING_DIFFICULTY  LUNG_CANCER  
0                    2                      1            0  


C:\Users\dell\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5123 - loss: 0.7016 - val_accuracy: 0.5250 - val_loss: 0.6967
Epoch 2/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5612 - loss: 0.6871 - val_accuracy: 0.5312 - val_loss: 0.6987
Epoch 3/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5735 - loss: 0.6846 - val_accuracy: 0.5250 - val_loss: 0.6951
Epoch 4/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5872 - loss: 0.6769 - val_accuracy: 0.5271 - val_loss: 0.6971
Epoch 5/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5615 - loss: 0.6822 - val_accuracy: 0.5437 - val_loss: 0.6989
Epoch 6/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5748 - loss: 0.6805 - val_accuracy: 0.5312 - val_loss: 0.6988
Epoch 7/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5596 - loss: 0.6810 - val_accuracy: 0.5271 - val_loss: 0.7013
Epoch 8/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5887 - loss: 0.6692 - val_accuracy: 0.5188 - val_

In [29]:
def get_valid_input(prompt, valid_func, error_msg):
    while True:
        try:
            value = valid_func(input(prompt))
            return value
        except ValueError:
            print(error_msg)

def predict_lung_cancer(model, scaler):
    print("\nNhập các thông tin sau để dự đoán nguy cơ ung thư phổi:")

    age = get_valid_input("Tuổi: ", lambda x: float(x) if 18 <= float(x) <= 80 else (_ for _ in ()).throw(ValueError()), 
                          "Tuổi không hợp lệ. Vui lòng nhập một số từ 18 đến 80.")
    
    gender = get_valid_input("Giới tính (F/M): ", lambda x: x.upper() if x.upper() in ['F', 'M'] else (_ for _ in ()).throw(ValueError()), 
                             "Giới tính không hợp lệ. Vui lòng nhập 'F' hoặc 'M'.").upper()
    
    air_pollution = get_valid_input("Mức độ ô nhiễm không khí (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                    "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    dust_allergy = get_valid_input("Mức độ dị ứng bụi (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                   "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    occupational_hazards = get_valid_input("Mức độ nguy hiểm nghề nghiệp (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                           "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    genetic_risk = get_valid_input("Mức độ rủi ro di truyền (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                   "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 10.")
    chronic_lung_disease = get_valid_input("Mức độ bệnh phổi mãn tính (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                           "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    balanced_diet = get_valid_input("Mức độ ăn uống cân bằng (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                    "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    obesity = get_valid_input("Mức độ béo phì (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                              "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 10.")
    smoking = get_valid_input("Mức độ hút thuốc (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                              "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    passive_smoker = get_valid_input("Mức độ hút thuốc thụ động (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                     "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    chest_pain = get_valid_input("Mức độ đau ngực (0-1): ", lambda x: float(x) if 0 <= float(x) <= 10 else (_ for _ in ()).throw(ValueError()),
                                 "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    coughing_of_blood = get_valid_input("Mức độ ho ra máu (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                        "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    weight_loss = get_valid_input("Mức độ giảm cân (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                  "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    shortness_of_breath = get_valid_input("Mức độ khó thở (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                          "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    wheezing = get_valid_input("Mức độ thở khò khè (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                               "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 10.")
    swallowing_difficulty = get_valid_input("Mức độ khó nuốt (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                            "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    clubbing_of_finger_nails = get_valid_input("Mức độ dùi trống của móng tay (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                               "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    frequent_cold = get_valid_input("Mức độ cảm lạnh thường xuyên (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                    "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    dry_cough = get_valid_input("Mức độ ho khan (0-1): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                                "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")
    snoring = get_valid_input("Mức độ ngáy (0-10): ", lambda x: float(x) if 0 <= float(x) <= 1 else (_ for _ in ()).throw(ValueError()),
                              "Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.")

    alcohol_consuming = get_valid_input("Bạn có uống rượu không? (Yes/No): ", lambda x: x.upper() if x.upper() in ['YES', 'NO'] else (_ for _ in ()).throw(ValueError()),
                                        "Thông tin về uống rượu không hợp lệ. Vui lòng nhập 'Yes' hoặc 'No'.").upper()
    
    fatigue = get_valid_input("Bạn có cảm thấy mệt mỏi không? (Yes/No): ", lambda x: x.upper() if x.upper() in ['YES', 'NO'] else (_ for _ in ()).throw(ValueError()),
                              "Thông tin về mệt mỏi không hợp lệ. Vui lòng nhập 'Yes' hoặc 'No'.").upper()

    # Chuyển đổi các giá trị nhị phân thành số
    gender = 0 if gender == 'F' else 1
    alcohol_consuming = 1 if alcohol_consuming == 'YES' else 0
    fatigue = 1 if fatigue == 'YES' else 0

    # Tạo một mảng đặc trưng từ các giá trị nhập vào
    input_features = np.array([[age, gender, air_pollution, alcohol_consuming, dust_allergy,
                                occupational_hazards, genetic_risk, chronic_lung_disease,
                                balanced_diet, obesity, smoking, passive_smoker, chest_pain,
                                coughing_of_blood, fatigue, weight_loss, shortness_of_breath,
                                wheezing, swallowing_difficulty, clubbing_of_finger_nails,
                                frequent_cold, dry_cough, snoring]])

    # Chuẩn hóa đặc trưng
    input_features = input_features[:, :14]  # Chỉ lấy 14 đặc trưng đầu tiên
    input_features = scaler.transform(input_features)

    # Dự đoán nguy cơ ung thư phổi
    prediction = model.predict(input_features)
    predicted_class = (prediction > 0.5).astype(int).flatten()

    if predicted_class[0] == 1:
        print("\nDự đoán: Bệnh nhân có nguy cơ cao bị ung thư phổi.")
    else:
        print("\nDự đoán: Bệnh nhân có nguy cơ thấp bị ung thư phổi.")



# Nhận dữ liệu từ người dùng và dự đoán
predict_lung_cancer(model, scaler)



Nhập các thông tin sau để dự đoán nguy cơ ung thư phổi:


Tuổi:  5


Tuổi không hợp lệ. Vui lòng nhập một số từ 18 đến 80.


Tuổi:  25
Giới tính (F/M):  F
Mức độ ô nhiễm không khí (0-1):  2


Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.


...

Mức độ không hợp lệ. Vui lòng nhập một số từ 0 đến 1.


Mức độ hút thuốc (0-1):  0
Mức độ hút thuốc thụ động (0-1):  0
Mức độ đau ngực (0-1):  0
Mức độ ho ra máu (0-1):  0
Mức độ giảm cân (0-1):  00
Mức độ khó thở (0-1):  0
Mức độ thở khò khè (0-1):  0
Mức độ khó nuốt (0-1):  0
Mức độ dùi trống của móng tay (0-1):  0
Mức độ cảm lạnh thường xuyên (0-1):  0
Mức độ ho khan (0-1):  0
Mức độ ngáy (0-10):  0
Bạn có uống rượu không? (Yes/No):  0


Thông tin về uống rượu không hợp lệ. Vui lòng nhập 'Yes' hoặc 'No'.


Bạn có uống rượu không? (Yes/No):  yes
Bạn có cảm thấy mệt mỏi không? (Yes/No):  yes


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step

Dự đoán: Bệnh nhân có nguy cơ thấp bị ung thư phổi.
